In [1]:
! pip install semantic-kernel -U

In [2]:
import os
import sys

import semantic_kernel as sk

In [3]:

kernel = sk.Kernel()

In [4]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [5]:
service_id = None

In [6]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_service(skaoai.AzureChatCompletion(deployment_name=deployment,endpoint=endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

In [7]:
from semantic_kernel.planners.basic_planner import BasicPlanner

In [8]:

planner = BasicPlanner(service_id)

In [9]:
base_skills_directory = '../../../plugins'

In [10]:
import APIPlugin.CustomAPIPlugin as custom_plugin

In [11]:

custom_plugin = kernel.import_plugin_from_object(custom_plugin.CustomAPIPlugin(), plugin_name="CustomAPI")  #.import_native_skill_from_directory(base_plugin , "APIPlugin")
writer_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "TranslatePlugin")

In [12]:
ask = """
Write email about  travellings tips based on  getting current weather in Guangzhou and translate it to Chinese"""

In [13]:
service_id

In [14]:

plan = await planner.create_plan(ask, kernel)

In [15]:
print(plan.generated_plan)

    {
        "input": "Travelling tips based on current weather in Guangzhou",
        "subtasks": [
            {"function": "CustomAPI.WeatherFunction", "args": {"city": "Guangzhou"}},
            {"function": "WriterPlugin.Tips"},
            {"function": "EmailPlugin.WeatherMail"},
            {"function": "TranslatePlugin.MultiLanguage", "args": {"language": "Chinese"}}
        ]
    }


In [16]:
results = await planner.execute_plan(plan, kernel)

In [17]:
results

'主题：即将出行的重要旅行和穿着建议\n\n亲爱的团队，\n\n希望这封邮件找到你们时一切都好。在我们准备即将的旅行时，我想分享一些重要的建议，以确保我们在旅行期间都能保持舒适和安全。\n\n1. 穿着浅色，宽松的衣服：浅色反射热量和阳光，宽松的衣服允许空气流通。这将帮助我们在户外活动期间保持凉爽和舒适。\n\n2. 坚持使用天然面料：棉和亚麻是透气的材料，这意味着它们允许一些空气接触到你的皮肤，帮助你保持凉爽。这些面料在我们即将访问的较暖和的气候中将特别有益。\n\n3. 保护你的眼睛和皮肤：戴上太阳镜，帽子或者帽子来保护你的眼睛免受阳光的伤害。别忘了涂抹防晒霜来保护你的皮肤免受有害紫外线的伤害。这一点至关重要，因为我们将在户外度过很多时间。\n\n4. 保持水分：随身携带水壶以保持水分。当我们忙于探索时，很容易忘记喝足够的水，但这对我们的健康和福祉至关重要。\n\n5. 不要忘记你的雨伞：雨伞可以保护你免受阳光和意外雨淋的伤害。这是一个简单的物品，但可以在我们的舒适度上产生很大的影响。\n\n请记住随时关注旅行条件。天气可能会迅速变化，对任何情况都要做好准备。\n\n让我们互相照顾，使这次旅行成为一个难忘和愉快的旅程。如果你有任何问题或需要进一步的信息，请随时提问。\n\n祝旅途平安，\n\n[你的名字]'